In [ ]:
import tensorflow as tf
import os

# Hide tf logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # or any {'0', '1', '2'}
# 0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs,
#     and 3 to additionally filter out ERROR logs.
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from pyDOE import lhs  #Latin Hypercube Sampling
import pandas as pd
import pickle

# generates same random numbers each time
np.random.seed(1234)
tf.random.set_seed(1234)

print("TensorFlow version: {}".format(tf.__version__))


$\frac{\partial p(x,t)}{\partial t} =
\frac{\partial}{\partial x}\left[\frac{m\omega^2}{\gamma} x \,p(x,t) + D \frac{\partial p(x,t)}{\partial x}\right]
\qquad A= \frac{m\omega^2}{\gamma}= \frac{k}{\gamma}$

# *Definitions and Data Prep*

## *Defining the domain and physical constants*

In [ ]:
from scipy import constants

Temperature = 300  # K
viscosity = 1e-3  # Kg / (m * s)
radius = 20e-10  # m
gamma = 6 * np.pi * viscosity * radius  # Kg / s
k = 3e-7  # Kg / s^2; 10GHz for frequency and 3e-26 Kg for mass
A = k / gamma  # s^-1
D = Temperature * constants.k / gamma  # m^2 / s
print(f"{gamma = } [Kg/s]")
print(f"{k = } [Kg/s^2]")
print(f"kB = {constants.k} [J/K]")
print(f"{A = :5.4e} [s^-1],\t{D = :4.4e} [m^2/s]")
# Change of units m -> µm
#                 s -> ms
# such that A ~ O(1), D ~ O(0.1)
A = A * 1e-3  # s^-1 -> ms^-1
D = D * 1e-3 * 1e+12  # m^2/s -> µm^2/ms
print(f"{A = :4.4e} [ms^-1],\t{D = :4.4e} [µm^2/ms]")
# Collocation points for every position and every time
x_lower = -0.1
x_upper = 0.25
N_x = 350
dx = (x_upper - x_lower) / N_x
x = np.linspace(x_lower, x_upper, N_x)  # µm line length
t_lower = 0
t_upper = 0.69 / A  # Typical Ornstein-Uhlenbeck time is ln(2) / A
N_steps = 200
t = np.linspace(t_lower, t_upper, N_steps)  # ms time interval
X, T = np.meshgrid(x, t)
# Max epochs in training
maxiter = 7500
# Total number of boundary conditions points
N_bc = int(N_steps * 0.45)
# Total number of initial condition points
N_ic = int(N_x * 0.45)
# Total number of collocation points
N_f = int(N_steps * N_x * 0.5)

## *Test Data*
We take the numerical solutions of the ```fplanck``` package as the test data to compare against the solution produced
by our PINN.

In [ ]:
# Initial Condition from imported numerical solution
real = pickle.load(open('./simulations/numerical_solution.sav', 'rb'))
psol=np.zeros((len(x),len(t)))
psol[:,0]=real[0]

X_p_test = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

# Domain bounds
low_bound = np.array([x_lower, t_lower])
up_bound = np.array([x_upper, t_upper])

p = psol.flatten('F')

## *Training and Validation Data*

The boundary and initial conditions serve as the training data for the PINN. We also select the collocation points
using *Latin Hypercube Sampling*.

We choose points of the domain that were not chosen for the training. We select the validation set to have the same size
as the training set.

In [ ]:
def training_and_valid_data(n_bc, n_ic, n_f):
    # Exception if 2N > Total domain size
    """ Initial Condition"""

    #Initial Condition -1 =< x =<1 and t = 0
    all_ic_x = np.vstack((X[0, :], T[0, :])).T
    all_ic_p = psol[:, 0].reshape(len(psol[:, 0]), 1)

    '''Boundary Conditions'''

    #Boundary Condition x = -1 and 0 =< t =<1
    bottomedge_x = np.vstack((X[:, 0], T[:, 0])).T
    #bottomedge_p = psol[-1, :].reshape(len(psol[-1, :]), 1) # Not needed in reflecting boundaries

    #Boundary Condition x = 1 and 0 =< t =<1
    topedge_x = np.vstack((X[:, -1], T[:, 0])).T
    #topedge_p = psol[0, :].reshape(len(psol[0, :]), 1) # Not needed in reflecting boundaries

    all_bc_x = np.vstack([bottomedge_x, topedge_x])
    # Reflecting conditions do not use the value of p
    #all_bc_p_train = np.vstack([ bottomedge_p, topedge_p])

    #choose random N_bc and N:ic points for training
    index_bc = np.random.choice(all_bc_x.shape[0], 2 * n_bc, replace=False)
    index_ic = np.random.choice(all_ic_x.shape[0], 2 * n_ic, replace=False)

    _x_bc_train = all_bc_x[index_bc[:len(index_bc)//2], :]
    _x_ic_train = all_ic_x[index_ic[:len(index_ic)//2], :]
    _p_ic_train = all_ic_p[index_ic[:len(index_ic)//2], :]

    _x_bc_valid = all_bc_x[index_bc[len(index_bc)//2 + 1:], :]
    _x_ic_valid = all_ic_x[index_ic[len(index_ic)//2 + 1:], :]
    _p_ic_valid = all_ic_p[index_ic[len(index_ic)//2 + 1:], :]

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points
    # N_f sets of tuples(x,t)
    all_f_x = low_bound + (up_bound - low_bound) * lhs(2, 2 * n_f)
    _x_f_train = all_f_x[:len(all_f_x)//2, :]
    _x_f_valid = all_f_x[len(all_f_x)//2 + 1:, :]
    # Do we select boundary and initial points also for f calculation?
    #   Only god knows
    #x_f_train = np.vstack((x_f_train, x_bc_train, x_ic_train))

    '''Normalization Instants'''
    # N_steps is the global variable for the time steps of the domain
    _x_norm_instant = np.vstack((X[int(N_steps / 2), :],
                                T[int(N_steps / 2), :])).T

    return _x_f_train, _x_bc_train, _x_ic_train, _p_ic_train, _x_norm_instant,\
           _x_f_valid, _x_bc_valid, _x_ic_valid, _p_ic_valid


In [ ]:
# Instantiating training and validation data
x_f_train, x_bc_train, x_ic_train, p_ic_train, x_norm_instants,\
    x_f_valid, x_bc_valid, x_ic_valid, p_ic_valid = training_and_valid_data(N_bc, N_ic, N_f)

# **PINN implementation**

## *Functions to save the training progress*

In [ ]:
extra_features = []
print("Training with extra features:", extra_features)
additional_constraints = []
print("Training with extra constraints:", additional_constraints)

def save_train_history(neural_network, name=None):
    history_filename = "./data/TRAIN_HISTORY_"
    if len(additional_constraints) != 0:
        for constrain in additional_constraints:
            history_filename = history_filename + constrain + "_"
    history_filename = history_filename + f"IC{N_ic}_BC{N_bc}_f{N_f}_t{t_upper:4.4f}_iter{maxiter}"
    if len(extra_features) != 0:
        for feat in extra_features:
            history_filename = history_filename + "_" + feat
    if name is not None:
        history_filename = history_filename + "_" + name
    print("Saving", history_filename, "...")
    neural_network.history["path"] = history_filename
    history = neural_network.get_training_history()
    pd.DataFrame(history).to_csv(history_filename + ".csv")


def save_model_weights(neural_network, name=None):
    model_filename = "./models/MODEL_WEIGHTS_"
    if len(additional_constraints) != 0:
        for constrain in additional_constraints:
            model_filename = model_filename + constrain + "_"
    model_filename = model_filename + f"IC{N_ic}_BC{N_bc}_f{N_f}_t{t_upper:4.4f}_iter{maxiter}"
    if len(extra_features) != 0:
        for feat in extra_features:
            model_filename = model_filename + "_" + feat
    if name is not None:
        model_filename = model_filename + "_" + name
    print("Saving", model_filename, "...")
    np.savetxt(model_filename + ".txt", neural_network.get_weights().numpy())

## *PINN class definition*
Initialization: ***Xavier***

In [ ]:
class PINN(tf.Module):
    def __init__(self,
                 layers,
                 do_print=True,
                 _additional_constraints=(),
                 a=10,
                 d=0.1,
                 f_regularization=1.0,
                 activation_function='tanh'):
        # The IC, BC, collocation points as well as p(IC points) and the domain upper/lower bounds are global variables,
        #     because they define the domain of the learning problem for our model,
        #     therefore they are required for our PINN class to be instantiated.
        for _global_variable in ['x_f_train', 'x_bc_train', 'x_ic_train', 'p_ic_train', 'x_norm_instants', 'low_bound',
                                'up_bound']:
            if _global_variable not in globals():
                raise ValueError("MissingGlobalVariable: " + _global_variable)
        super(PINN, self).__init__(name="PINN")
        self.layers = layers
        self.additional_constraints = _additional_constraints
        if activation_function == 'tanh':
            self.activation_function = tf.nn.tanh
        elif activation_function == 'swish':
            self.activation_function = tf.nn.swish
        else:
            raise ValueError("ActivationFunction:" + activation_function)
        self.a = a
        self.d = d
        self.f_regularization = f_regularization
        self.epoch = 0
        self.do_print = do_print
        self.save_training_after_n = 1000
        self.history = {"path": "",
                        "epoch": [],
                        "Total loss": [],
                        "IC loss": [],
                        "BC loss": [],
                        "f loss": [],
                        "Pr loss": [],
                        "Norm loss": [],
                        "Equi loss": []}
        self.W = []  #Weights and biases
        self.parameters = 0  #total number of parameters
        for i in range(len(layers) - 1):
            input_dim = layers[i]
            output_dim = layers[i + 1]
            #Xavier standard deviation
            std_dv = np.sqrt((2.0 / (input_dim + output_dim)))
            #weights = normal distribution * Xavier standard deviation + 0
            w = tf.random.normal([input_dim, output_dim], dtype='float64') * std_dv
            w = tf.Variable(w, trainable=True, name='w' + str(i + 1))
            b = tf.Variable(tf.cast(tf.zeros([output_dim]), dtype='float64'), trainable=True, name='b' + str(i + 1))
            self.W.append(w)
            self.W.append(b)
            self.parameters += input_dim * output_dim + output_dim

    def evaluate(self, subset):
        layer_input = (subset - low_bound) / (up_bound - low_bound)  # Normalization
        for i in range(len(self.layers) - 2):
            layer_output = tf.add(tf.matmul(layer_input, self.W[2 * i]), self.W[2 * i + 1])
            layer_input = self.activation_function(layer_output)
        layer_output = tf.add(tf.matmul(layer_input, self.W[-2]), self.W[-1])  # Regression: no activation to last layer
        return layer_output

    def get_weights(self):
        parameters_1d = []  # [.... W_i,b_i.....  ] 1d array
        for i in range(len(self.layers) - 1):
            w_1d = tf.reshape(self.W[2 * i], [-1])  #flatten weights
            b_1d = tf.reshape(self.W[2 * i + 1], [-1])  #flatten biases
            parameters_1d = tf.concat([parameters_1d, w_1d], 0)  #concat weights
            parameters_1d = tf.concat([parameters_1d, b_1d], 0)  #concat biases
        return parameters_1d

    def set_weights(self, parameters):

        for i in range(len(self.layers) - 1):
            shape_w = tf.shape(self.W[2 * i]).numpy()  # shape of the weight tensor
            size_w = tf.size(self.W[2 * i]).numpy()  #size of the weight tensor
            shape_b = tf.shape(self.W[2 * i + 1]).numpy()  # shape of the bias tensor
            size_b = tf.size(self.W[2 * i + 1]).numpy()  #size of the bias tensor
            pick_w = parameters[0:size_w]  #pick the weights
            self.W[2 * i].assign(tf.reshape(pick_w, shape_w))  # assign
            parameters = np.delete(parameters, np.arange(size_w), 0)  #delete
            pick_b = parameters[0:size_b]  #pick the biases
            self.W[2 * i + 1].assign(tf.reshape(pick_b, shape_b))  # assign
            parameters = np.delete(parameters, np.arange(size_b), 0)  #delete

    def set_training_history(self, path):
        history = pd.read_csv(path)
        self.history["epoch"] = list(history["epoch"])
        if self.history["epoch"] is not []:
            self.epoch = self.history["epoch"]
        self.history["Total loss"] = list(history["Total loss"])
        self.history["IC loss"] = list(history["IC loss"])
        self.history["BC loss"] = list(history["BC loss"])
        self.history["f loss"] = list(history["f loss"])
        self.history["Pr loss"] = list(history["Pr loss"])
        self.history["Norm loss"] = list(history["Norm loss"])
        self.history["Equi loss"] = list(history["Equi loss"])

    def get_training_history(self):
        return self.history

    def set_epoch(self, epoch):
        self.epoch = epoch

    def get_epoch(self):
        return self.epoch

    def set_pde_params(self, a, d):
        self.a = a
        self.d = d

    # Satisfy the IC
    def loss_ic(self, x_ic, p_ic):
        # Relative MSE
        return tf.reduce_mean(tf.square(p_ic - self.evaluate(x_ic))) / tf.reduce_sum(tf.square(p_ic))

    # Satisfy the reflecting boundary
    def loss_bc(self, boundary_points):
        variable_bc = tf.Variable(boundary_points, dtype='float64', trainable=False)
        x_bc = variable_bc[:, 0:1]
        t_bc = variable_bc[:, 1:2]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_bc)
            tape.watch(t_bc)
            tensor_bc = tf.stack([x_bc[:, 0], t_bc[:, 0]], axis=1)
            output_p_bc = self.evaluate(tensor_bc)
        p_x = tape.gradient(output_p_bc, x_bc)  #more efficient out of the context
        del tape
        flux = -1 * (self.a * x_bc * output_p_bc + self.d * p_x)
        return tf.reduce_mean(tf.square(flux))  # MSE_bc

    # Satisfy the PDE at the collocation points
    def loss_pde(self, collocation_points):
        variable_collocation = tf.Variable(collocation_points, dtype='float64', trainable=False)
        x_f = variable_collocation[:, 0:1]
        t_f = variable_collocation[:, 1:2]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_f)
            tape.watch(t_f)
            tensor_collocation = tf.stack([x_f[:, 0], t_f[:, 0]], axis=1)
            output_p_collocation = self.evaluate(tensor_collocation)
            p_x = tape.gradient(output_p_collocation, x_f)  #inside the context bc we need it for higher derivative
        p_t = tape.gradient(output_p_collocation, t_f)
        p_xx = tape.gradient(p_x, x_f)
        del tape
        f = p_t - self.a * output_p_collocation - self.a * x_f * p_x - self.d * p_xx
        return tf.reduce_mean(tf.square(f))  # MSE_f

    # Satisfy |probability| = 1 at some instants
    def loss_norm(self):
        o = self.evaluate(x_norm_instants)
        return tf.abs(tf.reduce_sum(o) * dx - 1.0)  # ME |norm - 1|

    # Must be Boltzmann distributed at t >> 1 with ß * m * w ^ 2 = A / D
    def loss_equi(self):
        # Typical time is ln(2) / A ≈ 0.69 / A
        t_large = 10 * (0.69 / self.a) * np.ones(256).reshape(256, 1)
        x_domain = np.linspace(low_bound[0], up_bound[0], 256).reshape(256, 1)
        x_at_large_t = tf.stack([x_domain[:, 0], t_large[:, 0]], axis=1)
        output = self.evaluate(x_at_large_t)
        boltzmann_dist = tf.exp(-1 * (self.a / (2 * self.d)) * x_domain ** 2)
        z = tf.reduce_sum(boltzmann_dist) * dx
        boltzmann_dist = boltzmann_dist / z
        # L2 norm (Boltzmann_dist - output)
        return tf.reduce_mean(tf.square(boltzmann_dist - output) * dx)

    # Satisfy p > 0 at IC and the collocation points
    def loss_prob(self, x_ic, x_f):
        o1 = self.evaluate(x_ic)
        o2 = self.evaluate(x_f)
        negatives = tf.where(tf.greater_equal(o1, 0.),
                             tf.zeros_like(o1),
                             o1)
        loss_pr = tf.abs(tf.reduce_mean(negatives))  # MSE (p < 0) at IC
        negatives = tf.where(tf.greater_equal(o2, 0.),
                             tf.zeros_like(o2),
                             o2)
        loss_pr = loss_pr + tf.abs(tf.reduce_mean(negatives))  # MSE (p < 0) at collocation
        return loss_pr

    def loss(self, x_ic, p_ic, x_bc, x_f):
        loss_ic = self.loss_ic(x_ic, p_ic)
        loss_bc = self.loss_bc(x_bc)
        loss_f = self.loss_pde(x_f)
        loss_prob = self.loss_prob(x_ic, x_f)
        loss_norm = self.loss_norm()
        loss_equi = self.loss_equi()
        loss = loss_ic + loss_bc + self.f_regularization * loss_f
        if "prob" in self.additional_constraints:
            loss = loss + loss_prob
        if "norm" in self.additional_constraints:
            loss = loss + loss_norm
        if "equi" in self.additional_constraints:
            loss = loss + loss_equi
        return loss, loss_ic, loss_bc, loss_f, loss_prob, loss_norm, loss_equi

    def optimizerfunc(self, parameters):
        self.set_weights(parameters)
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            total_loss, loss_ic, loss_bc, loss_f, loss_pr, loss_norm, loss_equi = self.loss(x_ic_train, p_ic_train,
                                                                                            x_bc_train, x_f_train)
            grads = tape.gradient(total_loss, self.trainable_variables)
        self.epoch += 1 # This has been an iteration of the training process.
        if self.do_print:
            tf.print(f"epoch: {self.epoch}", f"- Total: {total_loss:5.4e}", f"IC: {loss_ic:5.4e}",
                     f"BC: {loss_bc:5.4e}", f"f: {loss_f:5.4e}", f"Norm: {loss_norm:5.4e}", f"equi: {loss_equi:5.4e}")
        del tape
        grads_1d = []  #flatten grads
        for i in range(len(self.layers) - 1):
            grads_w_1d = tf.reshape(grads[2 * i], [-1])  #flatten weights
            grads_b_1d = tf.reshape(grads[2 * i + 1], [-1])  #flatten biases
            grads_1d = tf.concat([grads_1d, grads_w_1d], 0)  #concat grad_weights
            grads_1d = tf.concat([grads_1d, grads_b_1d], 0)  #concat grad_biases
        self.history["epoch"].append(self.epoch)
        self.history["Total loss"].append(float(total_loss))
        self.history["IC loss"].append(float(loss_ic))
        self.history["BC loss"].append(float(loss_bc))
        self.history["f loss"].append(float(loss_f))
        self.history["Pr loss"].append(float(loss_pr))
        self.history["Norm loss"].append(float(loss_norm))
        self.history["Equi loss"].append(float(loss_equi))
        if self.epoch % self.save_training_after_n == 0:
            save_train_history(self)
            save_model_weights(self)
        return total_loss.numpy(), grads_1d.numpy()



# *Grid Search with TensordBoard*

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/
from tensorboard.plugins.hparams import api as hp

for global_variable in ['x_f_valid', 'x_bc_valid', 'x_ic_valid', 'p_ic_valid', 'maxiter', 'A', 'D']:
    if global_variable not in globals():
        raise ValueError("MissingGlobalVariable: " + global_variable)

## *Hyperparameters to tune*
- Hidden Layers: $(20, 20, ...)$ and $(80, 70, ..., 10)$
- Activation function: $\tanh x$ or $x \text{S}(x)$
- PDE loss regularization: $(10^k)_{k=-3}^{1}$
- Type of training: standard training or transfer training

## Defining Hyperparameters

In [ ]:
HP_ARCHI_TYPE = hp.HParam('Architecture Type', hp.Discrete(['uniform', 'decreasing']))
HP_NUM_LAYERS = hp.HParam('Number of Layers', hp.Discrete([2, 4, 6, 8]))
HP_ACTIV_FUNC = hp.HParam('Activation', hp.Discrete(['tanh', 'swish']))
HP_REGU_PDE_L = hp.HParam('MSE(f) Regularization', hp.Discrete([0.001, 0.1, 1., 10.]))
HP_LEARN_TYPE = hp.HParam('Learning Type', hp.Discrete(['standard', 'transfer']))
METRIC = 'validation loss'

with tf.summary.create_file_writer('logs/').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_LAYERS, HP_ARCHI_TYPE, HP_ACTIV_FUNC, HP_REGU_PDE_L, HP_LEARN_TYPE],
    metrics=[hp.Metric(METRIC, display_name='1/Validation Loss')],
  )

## Model training for a Hyperparameter choice

In [ ]:
def standard_train(hyperparams):
    # Defining the PINN for the Hyperparameter choice
    our_layers = np.array([2])
    if hyperparams[HP_ARCHI_TYPE] == 'uniform':
        our_layers = np.append(our_layers, np.repeat(20, hyperparams[HP_NUM_LAYERS]))
    elif hyperparams[HP_ARCHI_TYPE] == 'decreasing':
        our_layers = np.append(our_layers, np.arange(start=10, stop=10*(hyperparams[HP_NUM_LAYERS]+1), step=10)[::-1])
    else:
        raise ValueError("ArchitectureType:" + hyperparams[HP_ARCHI_TYPE])
    our_layers = np.append(our_layers, np.array([1]))
    pinn = PINN(our_layers,
                f_regularization=hyperparams[HP_REGU_PDE_L],
                activation_function=hyperparams[HP_ACTIV_FUNC],
                a=A,
                d=D,
                _additional_constraints=additional_constraints)
    # Training the PINN
    init_params = pinn.get_weights().numpy()
    results = scipy.optimize.minimize(fun=pinn.optimizerfunc,
                                      x0=init_params,
                                      args=(),
                                      method='L-BFGS-B',
                                      jac=True, #fun is assumed to return the gradient along with the objective function
                                      options={'disp': None,
                                               'maxcor': 200,
                                               'ftol': 1 * np.finfo(float).eps,
                                               'gtol': 5e-8,
                                               'maxfun': 50000,
                                               'maxiter': max(maxiter - pinn.get_epoch(), 0),
                                               'iprint': -1,  #print update every 50 iterations
                                               'maxls': 50})
    # Measuring the validation set loss
    pinn.set_weights(results.x)
    validation_loss = pinn.loss(x_ic_valid, p_ic_valid, x_bc_valid, x_f_valid)[0]
    save_model_weights(pinn, name=f"GridSearch_VL{validation_loss:4.4e}")
    return validation_loss ** -1


def transfer_train(hyperparams):
    # Defining the PINN for the Hyperparameter choice
    our_layers = np.array([2])
    if hyperparams[HP_ARCHI_TYPE] == 'uniform':
        our_layers = np.append(our_layers, np.repeat(20, hyperparams[HP_NUM_LAYERS]))
    elif hyperparams[HP_ARCHI_TYPE] == 'decreasing':
        our_layers = np.append(our_layers, np.arange(start=10, stop=10*(hyperparams[HP_NUM_LAYERS]+1), step=10)[::-1])
    else:
        raise ValueError("ArchitectureType:" + hyperparams[HP_ARCHI_TYPE])
    our_layers = np.append(our_layers, np.array([1]))
    a = np.linspace(A/100, A, num=4)
    d = np.linspace(D/100, D, num=4)
    pinn = PINN(our_layers,
                f_regularization=hyperparams[HP_REGU_PDE_L],
                activation_function=hyperparams[HP_ACTIV_FUNC],
                a=a[0],
                d=d[0],
                _additional_constraints=additional_constraints)
    results = []
    previous_pinn_weigths = pinn.get_weights().numpy()
    for i in range(0, len(a)):
        # Transfer learning -> Initialize with the weights of the previous PINN trained for smaller PDE parameters
        pinn.set_pde_params(a[i], d[i])
        pinn.set_weights(previous_pinn_weigths)
        init_params = previous_pinn_weigths
        results = scipy.optimize.minimize(fun=pinn.optimizerfunc,
                                          x0=init_params,
                                          args=(),
                                          method='L-BFGS-B',
                                          jac=True, #fun is assumed to return the gradient along with the objective function
                                          options={'disp': None,
                                                   'maxcor': 200,
                                                   'ftol': 1 * np.finfo(float).eps,
                                                   'gtol': 5e-8,
                                                   'maxfun': 50000,
                                                   'maxiter': max(maxiter - pinn.get_epoch(), 0),
                                                   'iprint': -1,  #print update every 50 iterations
                                                   'maxls': 50})
        previous_pinn_weigths = results.x
    # Measuring the validation set loss
    pinn.set_weights(results.x)
    validation_loss = pinn.loss(x_ic_valid, p_ic_valid, x_bc_valid, x_f_valid)[0]
    save_model_weights(pinn, name=f"GridSearch_VL{validation_loss:4.4e}")
    return validation_loss ** -1




def run(run_dir, hyperparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hyperparams)  # record the values used in this trial
    if hyperparams[HP_LEARN_TYPE] == 'standard':
        validation_loss = standard_train(hyperparams)
    elif hyperparams[HP_LEARN_TYPE] == 'transfer':
        validation_loss = transfer_train(hyperparams)
    else:
        raise ValueError("LearningType:" + hyperparams[HP_LEARN_TYPE])
    tf.summary.scalar(METRIC, validation_loss, step=1)

## GridSearch proper

In [ ]:
session_num = 0
for archi_type in HP_ARCHI_TYPE.domain.values:
    for num_layers in HP_NUM_LAYERS.domain.values:
        for activ_func in HP_ACTIV_FUNC.domain.values:
            for regu_pde_l in HP_REGU_PDE_L.domain.values:
                for learn_type in HP_LEARN_TYPE.domain.values:
                    hparams = {
                        HP_ARCHI_TYPE: archi_type,
                        HP_NUM_LAYERS: num_layers,
                        HP_ACTIV_FUNC: activ_func,
                        HP_REGU_PDE_L: regu_pde_l,
                        HP_LEARN_TYPE: learn_type
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/' + run_name, hparams)
                    session_num += 1